<a href="https://colab.research.google.com/github/DanielDialektico/rag_agentes_langchain_curso/blob/main/notebooks/langchain_message_history.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://dialektico.com/wp-content/uploads/2023/03/MiniLogoW4.png" alt="Dialéktico Logo" />

Este pequeño tutorial pertenece al curso de RAG con Langchain al que puedes acceder mediante la siguiente URL: https://www.youtube.com/playlist?list=PLlWTv9_GeWd32stuEMWpYOnxiVxnXaU6q

Sigue los videos del curso para recibir instrucciones y contexto sobre la ejecución de este Notebook.

<br>

# Se instalan e importan las librerías

In [ ]:
!pip install langchain==0.3.20
!pip install langchain_deepseek==0.1.2

In [ ]:
import os
import warnings
from operator import itemgetter
from typing import List
from langchain_deepseek import ChatDeepSeek
from langchain_core.documents import Document
from langchain_core.messages import  SystemMessage, HumanMessage, ToolMessage, BaseMessage, AIMessage
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from langchain_core.runnables import (
    RunnableLambda,
    ConfigurableFieldSpec,
    RunnablePassthrough,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from google.colab import userdata

warnings.filterwarnings('ignore')

## Se añade valor de API key mediante un secreto

In [ ]:
# Se añade la API key como variable de ambiente desde un secreto en Colab.
os.environ["DEEPSEEK_API_KEY"] = userdata.get('DEEPSEEK_API_KEY')

## Se declara el modelo a utilizar

In [ ]:
# Se define el modelo y añaden valores de parámetros.
model = ChatDeepSeek(
      model="deepseek-chat",
      temperature=0,
      max_tokens=100
      )

<br>

# Añadir memoria al bot

Poder utilizar mensajes pasados como contexto en una conversación, es una característica que permite generar interacciones más naturales con el usuario. LangChain permite realizar esto mediante la adición de esta información al conjunto de preguntas y respuestas.

Para esto, se debe de defnir una clase para el manejor de historiales del chat

In [ ]:
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """Implementación de historias de mensajes en memoria local."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Añade una lista de mensajes al historial."""
        self.messages.extend(messages)

    def clear(self) -> None:
        """Limpia el historial de mensajes."""
        self.messages = []

# Se declara la lista donde se almacenarán las conversaciones.
store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

`store` es un diccionario global que almacena los historiales de chat por ID de sesión.

La clave del diccionario es el session_id, y el valor es una instancia de InMemoryHistory.

La función `get_by_session_id`:
* Recibe un session_id como parámetro.
* Si el session_id no existe en store, crea un nuevo historial (InMemoryHistory) y lo almacena en el diccionario.
* Devuelve el historial de chat asociado a esa sesión.

Se trata de una implementación simple en memoria que no persiste los datos, útil para pruebas o sesiones temporales.


In [ ]:
# Se añade un registro al historial.
history = get_by_session_id("1")
history.add_message(AIMessage(content="Hola mundo"))
print(store)

<br>

## Utilizando cadenas

Un aspecto importante del Lenguaje de Expresión de LangChain (LangChain Expression Language) es que los runnables pueden encadenarse en secuencias. La salida de la llamada .invoke() de un runnable anterior se pasa como entrada al siguiente runnable.

Esto se puede hacer utilizando el operador pipe (|) o el método más explícito .pipe(), que funciona de la misma manera.

El RunnableSequence resultante es en sí mismo un runnable, lo que significa que puede invocarse, transmitirse en streaming o seguir encadenándose como cualquier otro runnable.

In [ ]:
# Se crea una cadena prompt - modelo.

prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente experto en matemáticas, responde de manera breve y directa"),
    ("user", "{question}")
])

chain = prompt | model

chain

In [ ]:
# Se utiliza la cadena para generar una respuesta.

question = '¿Cuáles son los primeros 10 dígitos del número Pi?'
response = chain.invoke(question)
response.content

In [ ]:
# Se intenta obtener una respuesta basada en memoria.

question = '¿Cuáles son los siguientes 10?'
response = chain.invoke(question)
response.content

<br>

## Utilizando cadenas con memoria

In [ ]:
# Se crea una cadena con una variable para el historial.
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente experto en matemáticas, responde de manera breve y directa"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}"),
])

chain = prompt | model

chain

In [ ]:
# Se crea un runable que considere los mensajes pasados como contexto.

chain_with_history = RunnableWithMessageHistory(
    chain, # El runnable base (puede ser un modelo de lenguaje)
    get_by_session_id, # Función que recupera el historial de mensajes
    input_messages_key="question", # Clave en la entrada donde está la pregunta del usuario
    history_messages_key="history", # Clave en la entrada donde se almacenan los mensajes previos
)

In [ ]:
# Se añaden mensajes al historial, ahora con un nuevo ID.
response = chain_with_history.invoke({"question": "¿Cuáles son los primeros 10 dígitos del número Pi?"},
    config={"configurable": {"session_id": "2"}}
)

response.content

In [ ]:
# Se revisa el estado actual del almacenamiento.
store

In [ ]:
response = chain_with_history.invoke({"question": "¿Cuáles son los siguientes 10?"},
    config={"configurable": {"session_id": "2"}}
)

response.content

Más información en: https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html

<br>

Herramientas disponibles por LangChain: https://python.langchain.com/docs/integrations/tools/

Más información en: https://python.langchain.com/docs/concepts/tools/

<br>

In [ ]:
# Dialektico Machine learning practices © 2025 by Daniel Antonio García Escobar
# is licensed under CC BY-NC 4.0. To view a copy of this license,
# visit https://creativecommons.org/licenses/by-nc/4.0/

# Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International
# Public License